In [2]:
import pandas as pd
import os
import json
res_dir = '/Users/macbook/Downloads/infer2021'

test_ind_df = pd.read_csv('/Users/macbook/Downloads/test_ind.csv')
test_occ_df = pd.read_csv('/Users/macbook/Downloads/test_occ.csv')

with open(f'{res_dir}/industry_hierachy.json', 'r') as f:
    ind_hie = json.load(f)['sub-industry-label']
with open(f'{res_dir}/occupation_hierachy.json', 'r') as f:
    occ_hie = json.load(f)['sub-occupation-label']

ind_map = {sub_v: k for k, v in ind_hie.items() for sub_v in v}
occ_map = {sub_v: k for k, v in occ_hie.items() for sub_v in v}

gt_df = pd.read_csv(f'{res_dir}/gt/gt.csv')
# 14tskh9560001-1,ONLINE,1, 63,3,609,V3V,コーナン商事株式会社,ホームセンター,品出し
gt_df.columns = ['id', 'ocr', 'gender','age', 'position', 'sub_industry','sub_occupation','1', '2','3']
gt_df['sub_industry'] = gt_df['sub_industry'].str.strip()
gt_df['sub_occupation'] = gt_df['sub_occupation'].str.strip()
gt_df['industry'] = gt_df['sub_industry'].map(ind_map)
gt_df['occupation'] = gt_df['sub_occupation'].map(occ_map)
gt_df['id'] = gt_df['id'].str.upper()

In [3]:

def load_result(target):
    return pd.concat([
        pd.read_csv(f'{res_dir}/{target}/multicls_agp/result_correct.csv'),
        pd.read_csv(f'{res_dir}/{target}/multicls_agp/result_failed.csv'),
    ])

def load_result2():
    return pd.concat([
        pd.read_csv(f'/Users/macbook/Downloads/multicls_full/result_correct.csv'),
        pd.read_csv(f'/Users/macbook/Downloads/multicls_full/result_failed.csv'),
    ])
pred_df = load_result('industry')
pred_df.head()

,ID,Work place,Business,Job,Large label,Small label,Predicted large,Confidence large,Predicted small,Confidence small,Large correct,Small correct
0,14TSKH9560001-3,下山口児童館,学童保育,保育業務,P,853,P,0.999993,853,0.999996,True,True
1,CC01401010000009M-1,（株）葉山旭屋牛肉店,肉の販売及び食品の製造,食品の製造販売,I,583,I,0.995131,583,0.985901,True,True
2,14NEGD7851001-1,逗葉地域医療センター,総合病院,医療事務,P,831,P,0.999996,831,0.999886,True,True
3,14NEGD7851001-2,セイコーウオッチ株式会社,腕時計の製造,商品企画,E,323,E,0.996764,323,0.993488,True,True
4,CC01401010000022M-2,エステサロンＳＩＳ,エステ,エステティシャン,N,789,N,0.999957,789,0.999846,True,True


In [4]:
def export(pred_df, outfile):
    fail_df = pred_df[~pred_df["Small correct"]].groupby("Small label").size().to_frame('failed_count').reset_index()
    label_df = pred_df.groupby("Small label").size().to_frame('total_label').reset_index()
    stat_df = label_df.merge(fail_df, on='Small label', how='left')
    stat_df['failed_percent'] = stat_df['failed_count'] / stat_df['total_label']
    stat_df['accuracy'] = 1 - stat_df['failed_percent'] 
    stat_df.to_excel(f'/Users/macbook/Downloads/infer2021/{outfile}')
    s = stat_df['total_label'].sum()
    f = stat_df['failed_count'].sum()
    print(stat_df['accuracy'].mean())
    print(s, f, (s - f)/s)
    print(stat_df['failed_count'].sum())
    print()
    return stat_df

# export(load_result('industry'), 'multicls_industry_2020_stat.xlsx')
# export(load_result('occupation'), 'multicls_occupation_2020_stat.xlsx')

In [5]:
# export(load_result2(), 'mag_occupation_fulltest_stat.xlsx')
export(load_result2(), 'multicls_industry_fulltest_stat.xlsx')

0.7897267709860035
991258 122302 0.876619406854724
122302



,Small label,total_label,failed_count,failed_percent,accuracy
0,011,38456,737,0.019165,0.980835
1,013,3730,666,0.178552,0.821448
2,030,4061,87,0.021423,0.978577
3,040,1230,40,0.032520,0.967480
4,091,3569,435,0.121883,0.878117
...,...,...,...,...,...
247,981,7285,215,0.029513,0.970487
248,982,22449,678,0.030202,0.969798
249,C,676,147,0.217456,0.782544
250,D,84949,3557,0.041872,0.958128
